In [20]:
import spacy, coreferee


text = """
Jan Tarnowski, syn Jana Amora Iuniora i Barbary z Rożnowa, wnuczki Zawiszy Czarnego,
pochodził z wpływowej szlacheckiej rodziny Leliwitów Tarnowskich posiadającej status
senatorski. Wychowywał się na dworach kardynała Fryderyka Jagiellończyka oraz królów
Jana Olbrachta, Aleksandra i Zygmunta Starego. Otrzymał wszechstronne, klasyczne,
humanistyczne wykształcenie, także militarne. W związku ze śmiercią najstarszego brata
od 1514 zajmował się sprawami spadkowymi i gospodarowaniem rodzinnym majątkiem. Odtąd
też trwały: jego spór z siostrzeńcem Piotrem Kmitą i żale względem dworu.
"""

nlp = spacy.load("pl_core_news_lg")
nlp.add_pipe('coreferee')
doc = nlp(text)
doc._.coref_chains.print()

for chains in doc._.coref_chains:
    for chain in chains:
        for i in chain:
            print(doc[i], '->', doc._.coref_chains.resolve(doc[i]))

0: Jan(1), Wychowywał(30), Otrzymał(48), zajmował(69), jego(83)
Jan -> None
Wychowywał -> [Jan]
Otrzymał -> [Jan]
zajmował -> [Jan]
jego -> [Jan]


In [38]:
token_map = {}
for chains in doc._.coref_chains:
    mention_idx = chains[chains.most_specific_mention_index][0]
    mention = doc[mention_idx]
    for chain in chains:
        for i in chain:
            if i != mention_idx:
                token_map[i] = mention.text
                
output = ''

for token in doc:
    if token.i in token_map:
        if token.pos_ != 'VERB':
            output += token_map[token.i] + ' '
        else:
            output += token_map[token.i] + ' ' + token.text + token.whitespace_
    else:
        output += token.text + token.whitespace_

print(output)


Jan Tarnowski, syn Jana Amora Iuniora i Barbary z Rożnowa, wnuczki Zawiszy Czarnego,
pochodził z wpływowej szlacheckiej rodziny Leliwitów Tarnowskich posiadającej status
senatorski. Jan Wychowywał się na dworach kardynała Fryderyka Jagiellończyka oraz królów
Jana Olbrachta, Aleksandra i Zygmunta Starego. Jan Otrzymał wszechstronne, klasyczne,
humanistyczne wykształcenie, także militarne. W związku ze śmiercią najstarszego brata
od 1514 Jan zajmował się sprawami spadkowymi i gospodarowaniem rodzinnym majątkiem. Odtąd
też trwały: Jan spór z siostrzeńcem Piotrem Kmitą i żale względem dworu.

